### Scripts used to run different metagenome binning tools

In [ ]:
#!/bin/bash
#SBATCH -J "binning"
#SBATCH -p gpu
#SBATCH -A scc_users
#SBATCH -t 01-00:00:00
#SBATCH -N 1
#SBATCH -n 1
#SBATCH -c 24
#SBATCH -G 2
#SBATCH --mem=128G

DATAPATH="/scratch1/users/yazhini.a01/binning_benchmarking/strain_madness/megahit_assembly/pooled_assembly"
source ~/miniforge3/bin/activate
/usr/bin/time -v vamb --outdir ${DATAPATH}/vamb_results --fasta ${DATAPATH}/single_pooled_final.contigs.fa --bamfiles ${DATAPATH}/bamfiles/*.bam -m 1000 --minfasta 200000 --cuda -p 24

conda activate metabat2_env
/usr/bin/time -v metabat2 -i ${DATAPATH}/sorted_pooled_final.contigs.fa -a ${DATAPATH}/abundances_gf_sorted.tsv -o ${DATAPATH}/metabat2_results/metabat2_results -t 24 -m 1500
conda deactivate


conda activate mcdevol_env
/usr/bin/time -v /scratch1/users/yazhini.a01/binning_benchmarking/binning_tool/mcdevol/mcdevol.py -c ${DATAPATH}/sorted_pooled_final.contigs.fa -o ${DATAPATH}/mcdevol_results -a ${DATAPATH}/abundances_gf_sorted.tsv --abundformat metabat -n 24
conda deactivate

conda activate genomeface_env
/usr/bin/time -v genomeface -i ${DATAPATH}/sorted_pooled_final.contigs.fa -o ${DATAPATH}/genomeface_results -a ${DATAPATH}/abundances_gf_sorted.tsv -g marker_hits -m 1000
conda deactivate


conda activate metawrap-env
/usr/bin/time -v metawrap binning -o ${DATAPATH}/metawrap -t 24 -a ${DATAPATH}/sorted_pooled_final.contigs.fa --metabat2 --maxbin2 --concoct --interleaved /cbscratch/yazhini/binning_benchmarking/strain_madness/fastqfiles/coco_corrected/*.fastq
/usr/bin/time -v metawrap bin_refinement -o ${DATAPATH}/metawrap/bin_refined -t 24 -A ${DATAPATH}/metawrap/metabat2_bins/ -B ${DATAPATH}/metawrap/maxbin2_bins/ -C ${DATAPATH}/metawrap/concoct_bins/ -c 50 -x 10
conda deactivate


module load anaconda3/2021.05
conda activate comebin_env
/usr/bin/time -v /scratch1/users/yazhini.a01/binning_benchmarking/COMEBin/bin/run_comebin.sh -a ${DATAPATH}/single_pooled_final.contigs.fa -o ${DATAPATH}/comebin_results -p ${DATAPATH}/bamfiles -t 24
conda deactivate




